# Natural Language Project - Finding the Genre of Movie Plots - Group 11

## Project Setup
If you're running this project in **Google Colab**, make sure to execute the following commands to properly configure the environment.
(These steps are not required if you're running the project locally on your machine.)

In [ ]:
!git clone https://github.com/rspecker/NLP.git
%cd NLP

In [ ]:
# !pip install -r requirements.txt

In [ ]:
! pip install sentence-transformers

In [3]:
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# import project dependencies
import pandas as pd
from modellings.grid import classifiers
from preprocessing.embeddings import create_sentence_embeddings
from preprocessing.preproc import create_preprocesssed_dataset
from preprocessing.tfidf import create_train_data_tfidf
from utils import create_train_test_sets
from modelling.information_ret import score

## Data Import

In [8]:
# Import data and set column names
df = pd.read_table(
    'train.txt',
    names=['title', 'from', 'genre', 'director', 'plot']
    )

## Data Pre-Processing

In [9]:
# Split data into training and testing sets
x_train, x_test, y_train, y_test = create_train_test_sets(
    df, test_size=0.2, random_state=0, y_column='genre'
)

### TF-IDF
TF-IDF converts text into sparse vectors based on word importance across documents, without considering word order or context

In [ ]:
# create TF-IDF

### Feature-based Transfer Learning with Sentence Embeddings (Sentence-BERT)
Sentence-BERT generates dense, fixed-length embeddings that capture the semantic meaning of entire sentences, enabling more context-aware comparisons.

In [ ]:
# create sentence embeddings for testing data
x_train_sentence_embeddings = create_sentence_embeddings(
    sentences=x_train["plot"].to_list(),
    model="all-MiniLM-L6-v2")

In [11]:
# create sentence embeddings for testing data
x_test_sentence_embeddings = create_sentence_embeddings(
    sentences=x_test["plot"].to_list(),
    model="all-MiniLM-L6-v2")

## Training

### TF-IDF

### Feature-based Transfer Learning with Sentence Embeddings (Sentence-BERT)

In [ ]:
for model_name, (model, param_grid) in models.items():
    model_type="sentence_embeddings"
    # Perform GridSearchCV
    best_model, best_params, best_score = perform_grid_search(
        model, param_grid, model_type,model_name,
        x_train_sentence_embeddings, y_train)

    # Evaluate the model
    test_accuracy, classification_rep, cm = evaluate_model(
        best_model, x_test_sentence_embeddings, y_test)

    # Save the results
    save_results_to_file(
        model_type, model_name, best_params, best_score, test_accuracy, classification_rep)

    # Save confusion matrix as image
    save_confusion_matrix(model_type, model_name, cm, y_test)

In [ ]:
vectorizer = TfidfVectorizer(max_features=1000)
x_train_tfidf = vectorizer.fit_transform(x_train["plot"])
x_test_tfidf = vectorizer.transform(x_test["plot"])

model = MultinomialNB()
params, score = tune_hyperparameters(model, mnb_param_grid, x_train_tfidf, y_train)

final_model = MultinomialNB(alpha=params['alpha'],
                            fit_prior=params['fit_prior'])
final_model.fit(x_train_tfidf, y_train)

final_model.score(x_test_tfidf, y_test)
print(f"Test set accuracy: {final_model.score(x_test_tfidf, y_test)}")

model = SVC()
params, score = tune_hyperparameters(model, svc_param_grid, x_train_tfidf, y_train)

final_model = MultinomialNB(C=params['C'],
                            kernel=params['kernel'],
                            # gamma=params['gamma'],
                            # class_weight=params['class_weight']
                            )
final_model.fit(x_train_tfidf, y_train)

final_model.score(x_test_tfidf, y_test)
print(f"Test set accuracy: {final_model.score(x_test_tfidf, y_test)}")